In [ ]:
from capella2polarion.connectors import polarion_worker
from capella2polarion.converters import element_converter, converter_config, data_session
import dotenv
import os
import capellambse

Create a `.env` file with the following values:
- MODEL_PATH
- POLARION_PROJECT
- POLARION_HOST
- POLARION_PAT

In [ ]:
dotenv.load_dotenv()
model = capellambse.MelodyModel(os.environ.get("MODEL_PATH", ""))
worker = polarion_worker.CapellaPolarionWorker(
    polarion_worker.PolarionWorkerParams(
            os.environ.get("POLARION_PROJECT", ""),
            os.environ.get("POLARION_HOST", ""),
            os.environ.get("POLARION_PAT", ""),
            delete_work_items=False,
        )
)
worker.load_polarion_work_item_map()

In this example we want to test a jinja template for classes. We want to adjust nothing but the description field and we want to update all classes related to the class, too.

In [27]:
from capellambse.model.crosslayer import information

def add_class_incl_dependencies(cls: information.Class, classes: list[information.Class]):
    if cls in classes:
        return

    classes.append(cls)
    if cls.super:
        add_class_incl_dependencies(cls.super, classes)

    for property in cls.properties:
        if not (_type := getattr(property, "type")):
            continue

        if getattr(_type, "is_abstract", False):
            _type = property.abstract_type

        if isinstance(_type, information.Class):
            add_class_incl_dependencies(_type, classes)

In [28]:
classes = []
add_class_incl_dependencies(model.by_uuid("4ede9589-8b67-4b33-970d-810064b1cde6"), classes)
len(classes)

2

In [ ]:
class_config = converter_config.CapellaTypeConfig(
    "class",
    {
        "jinja_as_description": {
            "template_folder": "element_templates",
            "template_path": "class.html.j2",
        }
    },
    [],
)
serializer = element_converter.CapellaWorkItemSerializer(
    model,
    worker.polarion_data_repo,
    {
        cls.uuid: data_session.ConverterData(
            "pa",
            class_config,
            cls,
        )
        for cls in classes
    },
    False,
)


In [ ]:
wis = serializer.serialize_all()

In [ ]:
worker.client.project_client.work_items.update(wis)

Below you find an additional example for an ExchangeItem

In [ ]:
ei = model.by_uuid("f19d50b4-81d3-4fab-b8d3-55dfc9631444")
ei_config = converter_config.CapellaTypeConfig(
    "exchangeItem",
    {
        "jinja_as_description": {
            "template_folder": "element_templates",
            "template_path": "exchange_item.html.j2",
        }
    },
    [],
)
serializer = element_converter.CapellaWorkItemSerializer(
    model,
    worker.polarion_data_repo,
    {
        ei.uuid: data_session.ConverterData(
            "pa",
            ei_config,
            ei,
        )
    },
    False,
)

In [ ]:
wis = serializer.serialize_all()

In [ ]:
worker.client.project_client.work_items.update(wis)